In [26]:
from Bio import SeqIO
import pandas as pd

In [27]:
fasta_file = "final.fasta"  # Replace with the path to your FASTA file
records = SeqIO.parse(fasta_file, "fasta")

In [28]:
header_lines = []
sequences = []
with open("final.fasta", "r") as fasta_file:
    header = None
    sequence = ""
    for line in fasta_file:
        if line.startswith(">"):
            # This is a header line
            if header is not None:
                header_lines.append(header)
                sequences.append(sequence)
            header = line.strip()
            sequence = ""
        else:
            # This is sequence data
            sequence += line.strip()

    # Append the last record
    if header is not None:
        header_lines.append(header)
        sequences.append(sequence)
descriptions=[]
with open('final.fasta', 'r') as file:
    for line in file:
        if line.startswith('>'):
            # Extract the description part
            if line.startswith('>'):
                # Split the line by '|', and then further split by 'OS' to extract the description
                parts = line.split('|')
                if len(parts) >= 3:
                    description_up_to_OS = parts[2].split(' OS')[0].strip()
                    descriptions.append(description_up_to_OS)
# Create a DataFrame
data = {"Header Line": header_lines, "Sequence": sequences, 'Descriptions': descriptions}
df = pd.DataFrame(data)


In [29]:
# Assuming you have a DataFrame named df

# Access the full header (entire header line) of the first entry in the DataFrame
full_header = df.loc[0, "Header Line"]

# Print the full header
print("Full Header:")
print(full_header)


Full Header:
>PA2A6_CROHD | Crotalus | Acidic phospholipase A2 CH-E6' OS=Crotalus horridus OX=35024 PE=1 SV=1


In [30]:
# Create a new column "OS" in the DataFrame by extracting the OS information until "OX" from the "Header Line"
df['Species'] = df['Header Line'].str.extract(r'OS=(.*?)(?= OX)')
df['Species']=df['Species'].str.lower()
# Print the DataFrame to see the new "OS" column
df

,Header Line,Sequence,Descriptions,Species
0,>PA2A6_CROHD | Crotalus | Acidic phospholipase...,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 CH-E6',crotalus horridus
1,>PA2A_CROAD | Crotalus | Acidic phospholipase ...,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 beta,crotalus adamanteus
2,>PA2A_CROAT | Crotalus | Acidic phospholipase ...,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2,crotalus atrox
3,>PA2AA_CROVV | Crotalus | Acidic phospholipase...,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 Cvv-E6a,crotalus viridis viridis
4,>PA2AD_CROOA | Crotalus | Acidic phospholipase...,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 CoaPLA2,crotalus oreganus abyssus
...,...,...,...,...
246,>A0A6P5JCK6_PHACI | Phascolarctos | Phospholip...,VEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGWGGHGRPQDATDRC...,Phospholipase A2,phascolarctos cinereus
247,>A0A1S3GT82_DIPOR | Dipodomys | Phospholipase ...,MRTLWIVAVLL----LGVEGSLVQFEMMIMEVAKRSGLLWYSAYGC...,Phospholipase A2,dipodomys ordii
248,>PA2HB_BOTLC | Bothrops | Basic phospholipase ...,SLVQFEMMIMEVAKRSGLLWYSAYGCYCGWGGHGRPQDATDRCCFV...,Basic phospholipase A2 homolog blK-PLA2,bothrops leucurus
249,>PA2A2_BITNA | Bitis | Acidic phospholipase A2...,LVQFEMMIMEVAKRSGLLWYSAYGCYCGWGGHGRPQDATDRCCFVH...,Acidic phospholipase A2 CM-II,bitis nasicornis


In [31]:
from Bio.Seq import Seq
sequences=[Seq(seq) for seq in df['Sequence']]

In [32]:
from Bio.Align.Applications import ClustalOmegaCommandline

# Create a temporary FASTA file containing the sequences
with open("input_sequences.fasta", "w") as fasta_file:
    for i, seq in enumerate(sequences):
        fasta_file.write(f">Seq{i}\n{seq}\n")

# Define the output file path
output_file = "output_alignment.fasta"

# Create a command-line instance for Clustal Omega
clustalomega_cline = ClustalOmegaCommandline(infile="input_sequences.fasta", outfile=output_file, outfmt="fasta")

# Run Clustal Omega
stdout, stderr = clustalomega_cline()

In [33]:
sequences = {}
current_seq = ""

    # Open and read the FASTA file
with open('output_alignment.fasta', 'r') as file:
    for line in file:
        line = line.strip()
            # Check if the line is a header line
        if line.startswith('>'):
            if current_seq:
                sequences[header] = current_seq
                current_seq = ""
            header = line[1:]
        else:
            current_seq += line
        # Add the last sequence
    if current_seq:
        sequences[header] = current_seq

In [34]:
df1 = pd.DataFrame(list(sequences.items()), columns=['Header', 'Sequence'])
df['Aligned']=df1['Sequence']
df.head()

,Header Line,Sequence,Descriptions,Species,Aligned
0,>PA2A6_CROHD | Crotalus | Acidic phospholipase...,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 CH-E6',crotalus horridus,MRTLWIVAVLLLG-----VEGSLVQFEMMIMEVAKRSGLLWYSAYG...
1,>PA2A_CROAD | Crotalus | Acidic phospholipase ...,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 beta,crotalus adamanteus,MRTLWIVAVLLLG-----VEGSLVQFEMMIMEVAKRSGLLWYSAYG...
2,>PA2A_CROAT | Crotalus | Acidic phospholipase ...,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2,crotalus atrox,MRTLWIVAVLLLG-----VEGSLVQFEMMIMEVAKRSGLLWYSAYG...
3,>PA2AA_CROVV | Crotalus | Acidic phospholipase...,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 Cvv-E6a,crotalus viridis viridis,MRTLWIVAVLLLG-----VEGSLVQFEMMIMEVAKRSGLLWYSAYG...
4,>PA2AD_CROOA | Crotalus | Acidic phospholipase...,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 CoaPLA2,crotalus oreganus abyssus,MRTLWIVAVLLLG-----VEGSLVQFEMMIMEVAKRSGLLWYSAYG...


In [35]:
df.drop_duplicates(subset='Sequence', keep='first',inplace=True)

In [36]:
df=df.drop('Header Line',axis=1)
df.head()

,Sequence,Descriptions,Species,Aligned
0,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 CH-E6',crotalus horridus,MRTLWIVAVLLLG-----VEGSLVQFEMMIMEVAKRSGLLWYSAYG...
11,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 Tgc-E6,trimeresurus gracilis,MRTLWIVAVLL-----LGVEGSLVQFEMMIMEVAKRSGLLWYSAYG...
13,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 S1E6-c,calloselasma rhodostoma,MRTLWIVAVLL-----LGVEGSLVQFEMMIMEVAKRSGLLWYSAYG...
24,SLVQFEMMIMEVAKRSGLLWYSAYGCYCGWGGHGRPQDATDRCCFV...,Acidic phospholipase A2,deinagkistrodon acutus,---------------------SLVQFEMMIMEVAKRSGLLWYSAYG...
26,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 DE-I,ovophis okinavensis,MRTLWIVAVLL-----LGVEGSLVQFEMMIMEVAKRSGLLWYSAYG...


In [41]:
df.drop_duplicates(subset="Species", keep='first', inplace=True)

In [44]:
indexes_to_exclude=[209,216,226,228,238,245,246,247]
df_filtered = df.drop(indexes_to_exclude)
df_filtered

,Sequence,Descriptions,Species,Aligned
0,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 CH-E6',crotalus horridus,MRTLWIVAVLLLG-----VEGSLVQFEMMIMEVAKRSGLLWYSAYG...
11,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 Tgc-E6,trimeresurus gracilis,MRTLWIVAVLL-----LGVEGSLVQFEMMIMEVAKRSGLLWYSAYG...
13,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 S1E6-c,calloselasma rhodostoma,MRTLWIVAVLL-----LGVEGSLVQFEMMIMEVAKRSGLLWYSAYG...
24,SLVQFEMMIMEVAKRSGLLWYSAYGCYCGWGGHGRPQDATDRCCFV...,Acidic phospholipase A2,deinagkistrodon acutus,---------------------SLVQFEMMIMEVAKRSGLLWYSAYG...
26,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Acidic phospholipase A2 DE-I,ovophis okinavensis,MRTLWIVAVLL-----LGVEGSLVQFEMMIMEVAKRSGLLWYSAYG...
35,SLVQFEMMIMEVAKRSGLLWYSAYGCYCGWGGHGRPQDATDRCCFV...,Acidic phospholipase A2 A',gloydius halys,---------------------SLVQFEMMIMEVAKRSGLLWYSAYG...
40,SLVQFEMMIMEVAKRSGLLWYSAYGCYCGWGGHGRPQDATDRCCFV...,Acidic phospholipase A2,gloydius ussuriensis,---------------------SLVQFEMMIMEVAKRSGLLWYSAYG...
44,LVQFEMMIMEVAKRSGLLWYSAYGCYCGWGGHGRPQDATDRCCFVH...,Acidic phospholipase A2 4,trimeresurus gramineus,----------------------LVQFEMMIMEVAKRSGLLWYSAYG...
46,SLVQFEMMIMEVAKRSGLLWYSAYGCYCGWGGHGRPQDATDRCCFV...,Basic phospholipase A2 APP-D49,agkistrodon piscivorus piscivorus,---------------------SLVQFEMMIMEVAKRSGLLWYSAYG...
55,MRTLWIVAVLLLGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGW...,Neutral phospholipase A2 ammodytin I2,vipera ammodytes ammodytes,MRTLWIVAVLLLG-----VEGSLVQFEMMIMEVAKRSGLLWYSAYG...


In [48]:
fasta_file = "aligned_sequences.fasta"

with open(fasta_file, "w") as handle:
    for idx, row in df_filtered.iterrows():
        sequence = row["Aligned"]
        species = row["Species"]
        record = SeqIO.SeqRecord(Seq(sequence), id=species, description="")
        SeqIO.write(record, handle, "fasta")

# Step 2: Create a Nexus file using the generated FASTA file
nexus_file = "aligned_sequences.nex"
nexus_data = """#NEXUS
BEGIN DATA;
DIMENSIONS NTAX={ntax} NCHAR={nchar};
FORMAT DATATYPE=protein MISSING=? GAP=-;
MATRIX
{matrix}
;
END;
"""

ntax = len(df_filtered)
nchar = len(df_filtered["Aligned"][0])  # Assuming all sequences have the same length
matrix = []

with open(fasta_file, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        header = record.description
        sequence = str(record.seq)
        matrix.append(f"{header} {sequence}")
nexus_data = nexus_data.format(ntax=ntax, nchar=nchar, matrix="\n".join(matrix))

with open(nexus_file, "w") as handle:
    handle.write(nexus_data)

print(f"Nexus file '{nexus_file}' has been created.")

Nexus file 'aligned_sequences.nex' has been created.


In [46]:
record

SeqRecord(seq=Seq('MRTLWIVAVL----L-LGVEGSLVQFEMMIMEVAKRSGLLWYSAYGCYCGWGGH...EPC'), id='oryctolagus', name='oryctolagus', description='oryctolagus cuniculus', dbxrefs=[])